In [2]:
import sys
import xarray as xr
import numpy as np
import torch
import os
sys.path.append("/home/akazemi3/Desktop/MB_Lab_Project/") 

import random 
from sklearn.decomposition import PCA
from tqdm import tqdm 
import pickle 

ACTIVATIONS_PATH = '/data/atlas/activations'
NEURAL_DATA_PATH = '/data/atlas/neural_data'
DATASET = 'naturalscenes_zscored_processed'

PATH_TO_NSD_SHARED_IDS = '/data/atlas/neural_data/nsd_shared_ids'
file = open(PATH_TO_NSD_SHARED_IDS, 'rb')
SHARED_IDS = pickle.load(file)


import warnings
warnings.filterwarnings('ignore')

import scipy
from sklearn.preprocessing import scale
from tools.loading import *
from sklearn.linear_model import Ridge
from analysis.neural_data_regression.tools.scorers.function_types import Regression
from analysis.neural_data_regression.tools.regression import *


In [3]:
import numpy as np
import scipy.linalg
from sklearn.linear_model import RidgeCV

def RR_looe_demo(X_training, Y_training, alphas):
    """ Implements the ridge regression SVD-based LOOE estimate
    from http://cbcl.mit.edu/publications/ps/MIT-CSAIL-TR-2007-025.pdf, page 6
    args:
    X_training (np.ndarray) n_obs x n_features
    Y_training (np.ndarray) n_obs x n_targets
    alpha (iterable) n_alphas
    returns
    np.ndarray ( n_targets x n_alphas)
    ! assumes the columns of X_training and Y_training are centered
    """

    n_training = X_training.shape[0]
    n_targets =  Y_training.shape[1]
    n_alphas = len(alphas)

    [U,s,Vh] = scipy.linalg.svd(X_training,full_matrices=False,overwrite_a=False)
    del Vh, X_training

    I = np.eye(n_training)
    columnwise_mean_squared=lambda x: (x**2).mean(axis=0)

    looe = np.ones((n_targets,n_alphas))*np.nan

    for i_alpha, alpha in enumerate(alphas):
        # rewriting the LOOE equation from page 6 as AY/diag(A)

        #S_bar=np.diag(1/(s**2+alpha)-1/alpha)
        S_bar=np.diag(-(s**2)/(alpha*s**2+alpha**2)) # equivalent to the previous line but less prone to underflow when s is very small
        A=U@S_bar@U.T+I/alpha

        looe[:,i_alpha]=columnwise_mean_squared(
                            (A@Y_training)
                            / np.reshape(np.diag(A),(-1,1)))
    return looe

In [4]:
def load_nsd_data(mode: str, subject: int, region: str, return_ids: bool = True) -> torch.Tensor:
        
        """
        
        Loads the neural data from disk for a particular subject and region.


        Parameters
        ----------
        mode:
            The type of neural data to load ('shared' or 'unshared')
            
        subject:
            The subject number 
        
        region:
            The region name
            
        return_ids: 
            Whether the image ids are returned 
        

        Returns
        -------
        A Tensor of Neural data, or Tensor of Neural data and stimulus ids
        
        """
        
        file_name = f'subject_{subject}_{region}'
        file_path = os.path.join(NEURAL_DATA_PATH,DATASET,file_name)

        if mode == 'unshared':
            file_ext = '_unshared_new.nc'

        elif mode == 'shared':
            file_ext = '_shared_new.nc'

        data = xr.open_dataset(file_path + file_ext)
        ids = list(data.stimulus_id.values)
        data = torch.Tensor(data['x'].values).transpose(0,1)
        
        if return_ids:
            return ids, data
        else:
            return data
           
        
        
            
def filter_activations(data: xr.DataArray, ids: list) -> torch.Tensor:
            
        """
    
        Filters model activations using image ids.


        Parameters
        ----------
        data:
            Model activation data
            
        ids:
            image ids
        

        Returns
        -------
        A Tensor of model activations filtered by image ids
        
        """
        
        data = data.set_index({'presentation':'stimulus_id'})
        activations = data.sel(presentation=ids)
        activations = activations.sortby('presentation', ascending=True)

        return torch.Tensor(activations.values)
            


In [19]:
model_name = 'model_3L_mp_1000'
activations_identifier = 'model_3L_mp_1000' + '_naturalscenes_zscored_processed'
region = 'V4'
alphas = [10,100,1000]   
activations_data = xr.open_dataarray(os.path.join(ACTIVATIONS_PATH,activations_identifier))         


ds = xr.Dataset(data_vars=dict(r_value=(["r_values"], [])),
                            coords={'subject': (['r_values'], []),
                                    'region': (['r_values'], [])
                                     })

l = []

for alpha in alphas:
    
    regression_model = Ridge(alpha = alpha)
    for subject in range(8):

            print('subject: ',subject)

            ids, y = load_nsd_data(mode ='unshared',
                                   subject = subject,
                                   region = region,
                                   return_ids = True)


                
            idx = np.random.randint(0,len(ids),35000)
            sampled_ids = np.array(ids)[idx]

            X_sample = filter_activations(data = activations_data,
                                   ids = sampled_ids)

            y_sample = y[idx]




            y_true, y_predicted = regression_cv(x=X,
                                                y=y,
                                                model=regression_model,
                                                save_betas=False,
                                                scores_identifier='')
            r =  torch.stack(
               [pearson_r(y_true_, y_predicted_) for y_true_, y_predicted_ in zip(y_true, y_predicted)]
            ).mean(dim=0)


            print(r.mean())
            subject_list = [subject for i in range(len(r))]
            region_list = [region for i in range(len(r))]

            ds_tmp = xr.Dataset(data_vars=dict(r_value=(["r_values"], r)),
                                        coords={'subject': (['r_values'], subject_list),
                                                'region': (['r_values'], region_list)
                                                 })

            ds = xr.concat([ds,ds_tmp],dim='r_values')
    l.append(ds)

subject:  0


NameError: name 'X' is not defined

In [18]:
y = np.array(y)
y[idx]

array([[ 4.0453750e-01, -9.1948675e-04,  6.9110566e-01, ...,
         1.8597763e+00, -2.4656360e-01, -5.5316508e-01],
       [ 5.1491678e-01, -2.1221349e-01, -2.3578423e-01, ...,
        -3.0695957e-01, -1.4389749e+00,  3.3256328e-01],
       [-5.7884341e-01, -2.4045575e-01, -3.5428324e-01, ...,
        -1.7558341e-01, -1.3365078e+00, -5.8807856e-01],
       ...,
       [-2.7613586e-01, -3.0635437e-01, -4.3707019e-01, ...,
         6.2626410e-01, -1.7977032e-01,  1.2254518e-01],
       [ 7.2396898e-01,  8.1601435e-01,  1.4000392e-01, ...,
        -2.1959125e-01, -8.5984707e-01,  2.1869144e-01],
       [ 1.9891516e+00, -8.0425519e-01, -6.4160275e-01, ...,
        -1.7973989e+00, -6.1999851e-01, -7.5297761e-01]], dtype=float32)

In [17]:
np.array(sampled_ids)

array(['image46133', 'image51137', 'image32437', ..., 'image37056',
       'image52179', 'image06768'], dtype='<U10')

In [10]:
list(sampled_ids

array(['image42504', 'image68844', 'image40959', ..., 'image19682',
       'image28680', 'image12596'], dtype='<U10')

In [6]:
idx

array([6513, 4232, 2535, ..., 1443, 5372, 5107])

In [9]:
for i in range(len(l)):
    print(alphas[i], l[i].r_value.mean())

10 <xarray.DataArray 'r_value' ()>
array(0.16528444)
100 <xarray.DataArray 'r_value' ()>
array(0.17630017)
1000 <xarray.DataArray 'r_value' ()>
array(0.17537826)
